In [1]:
from name_mappings import (
    balance_sheet_mapping,
    income_statement_mapping,
    cash_flow_mapping,
)
from edgar_functions import save_dataframe_to_csv
from edgar_functions_V2 import *
import pandas as pd
from datetime import datetime

ticker = "WSM"
cik = get_cik(ticker)

In [2]:
submission_json = get_submission_data_for_ticker(ticker)
filings_df = get_submission_data_for_ticker(ticker, only_filings_df=True)

In [3]:
income_mapping = income_statement_mapping.income_mapping
balance_mapping = balance_sheet_mapping.balance_mapping
cash_mapping = cash_flow_mapping.cash_mapping

In [4]:
def process_and_aggregate_annual_data(ticker):
    cik = get_cik(ticker)
    frames_to_include = [
        "CY2007",
        "CY2008",
        "CY2009",
        "CY2010",
        "CY2011",
        "CY2012",
        "CY2013",
        "CY2014",
        "CY2015",
        "CY2016",
        "CY2017",
        "CY2018",
        "CY2019",
        "CY2020",
        "CY2021",
        "CY2022",
        "CY2023",
    ]
    filings_df = get_submission_data_for_ticker(ticker, only_filings_df=True)
    full_facts = get_annual_edgar_facts_from_downloaded_json(cik)
    full_facts["end"] = pd.to_datetime(full_facts["end"])
    full_facts["filed"] = pd.to_datetime(full_facts["filed"])
    pivot_table = pd.pivot_table(
        full_facts, index=["end", "frame"], columns=["fact"], values=["val"]
    )

    # Filter DataFrame based on frames_to_include
    filtered_df = pivot_table.loc[
        pivot_table.index.get_level_values(1).isin(frames_to_include)
    ]

    # Get unique dates
    unique_dates = filtered_df.index.get_level_values(0).unique()

    # Filter DataFrame again based on unique_dates
    final_filtered_df = pivot_table.loc[(unique_dates, slice(None)), :].sort_index(
        level=1, ascending=True
    )
    final_filtered_df.columns = final_filtered_df.columns.droplevel(0)

    # Insert additional columns
    final_filtered_df.insert(
        0, "Year", final_filtered_df.index.get_level_values(1).str[2:6]
    )
    final_filtered_df.insert(
        1, "Period End", final_filtered_df.index.get_level_values(0)
    )

    # Group by 'Year' and take the first entry for each group
    aggregated_df = final_filtered_df.groupby("Year").first()

    # Calculate extended filing dates
    desired_length = len(aggregated_df.index)
    filing_dates = filings_df[filings_df["form"] == "10-K"]["filingDate"]
    filing_dates = pd.to_datetime(filing_dates)
    average_time_delta = filing_dates.diff().dropna().mean()
    current_date = filing_dates.min()

    extended_dates = []
    for _ in range(desired_length - len(filing_dates)):
        next_date = current_date + average_time_delta
        extended_dates.append(next_date)
        current_date = next_date

    # Insert 'Filing Date' column
    extended_dates_series = pd.Series(extended_dates, name="Filing Date")
    final_extended_filing_dates = (
        pd.concat([filing_dates, extended_dates_series])
        .sort_values()
        .reset_index(drop=True)
    )
    final_extended_filing_dates = final_extended_filing_dates.dt.strftime("%Y-%m-%d")
    aggregated_df.insert(1, "Filing Date", final_extended_filing_dates.values)
    aggregated_df["Filing Date"] = pd.to_datetime(aggregated_df["Filing Date"])

    # Convert all column names to strings and drop NaN columns
    aggregated_df.columns = aggregated_df.columns.map(str)
    aggregated_df = aggregated_df.dropna(axis=1, how="all")

    return aggregated_df

In [5]:
processed_df = process_and_aggregate_annual_data(ticker)
processed_df

fact,Period End,Filing Date,AccountsPayableCurrent,AccruedIncomeTaxesCurrent,AccruedLiabilitiesCurrent,AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment,AccumulatedOtherComprehensiveIncomeLossForeignCurrencyTranslationAdjustmentNetOfTax,AccumulatedOtherComprehensiveIncomeLossNetOfTax,AdditionalPaidInCapital,AdjustmentsRelatedToTaxWithholdingForShareBasedCompensation,...,UnrecognizedTaxBenefitsDecreasesResultingFromSettlementsWithTaxingAuthorities,UnrecognizedTaxBenefitsIncomeTaxPenaltiesAndInterestAccrued,UnrecognizedTaxBenefitsIncreasesResultingFromCurrentPeriodTaxPositions,UnrecognizedTaxBenefitsIncreasesResultingFromPriorPeriodTaxPositions,UnrecognizedTaxBenefitsInterestOnIncomeTaxesAccrued,UnrecognizedTaxBenefitsReductionsResultingFromLapseOfApplicableStatuteOfLimitations,UnrecognizedTaxBenefitsThatWouldImpactEffectiveTaxRate,VariableLeaseCost,WeightedAverageNumberOfDilutedSharesOutstanding,WeightedAverageNumberOfSharesOutstandingBasic
Year,,,,,,,,,,,,,,,,,,,,,
2008,2009-02-01,2009-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"106,880,000","105,530,000"
2009,2010-01-31,2010-04-01,"188,241,000","48,260,000","107,710,000",NaN,"10,387,000",NaN,"448,848,000","-3,621,000",...,"329,000",NaN,"1,029,000","655,000",NaN,"1,553,000","10,594,000",NaN,"107,373,000","105,763,000"
2010,2011-01-30,2011-03-31,"227,963,000","41,997,000","122,440,000","1,127,493,000","12,990,000",NaN,"466,885,000","17,918,000",...,"1,701,000",NaN,"821,000",0,"4,062,000","2,807,000","7,812,000",NaN,"109,522,000","106,956,000"
2011,2012-01-29,2012-03-29,"218,329,000","22,435,000","111,774,000","1,199,805,000","12,590,000",NaN,"478,720,000","11,656,000",...,"2,070,000",NaN,"1,329,000","379,000","3,983,000","864,000","6,738,000",NaN,"106,582,000","104,352,000"
2012,2013-02-03,2013-04-04,"259,162,000","41,849,000","120,632,000","1,282,905,000","13,633,000",NaN,"503,616,000","18,637,000",...,"1,069,000","2,508,000","2,188,000","936,000","2,508,000","2,917,000","6,101,000",NaN,"101,051,000","99,266,000"
2013,2014-02-02,2014-04-03,"404,791,000","49,365,000","138,181,000","1,382,475,000","6,524,000",NaN,"522,595,000","18,096,000",...,"170,000","2,231,000","3,351,000","328,000",NaN,"1,692,000","7,202,000",NaN,"98,765,000","96,669,000"
2014,2015-02-01,2015-04-02,"397,037,000","32,488,000","136,012,000","1,386,465,000","-2,548,000","-2,548,000","527,261,000",NaN,...,"1,144,000","2,412,000","3,093,000","2,007,000",NaN,"224,000","9,602,000",NaN,"95,200,000","93,634,000"
2015,2016-01-29,2016-03-31,"447,412,000","67,052,000","127,122,000","1,454,307,000",NaN,"-10,616,000","541,307,000",NaN,...,"2,912,000","2,649,000","2,765,000","101,000",NaN,"682,000","8,948,000",NaN,"92,102,000","90,787,000"
2016,2017-01-29,2017-03-30,"453,710,000","23,245,000","130,187,000","1,586,039,000",NaN,"-9,903,000","556,928,000",NaN,...,"714,000","2,882,000","11,772,000","3,456,000",NaN,"1,122,000","21,134,000",NaN,"89,462,000","88,594,000"


In [6]:
processed_df.columns

Index(['Period End', 'Filing Date', 'AccountsPayableCurrent',
       'AccruedIncomeTaxesCurrent', 'AccruedLiabilitiesCurrent',
       'AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment',
       'AccumulatedOtherComprehensiveIncomeLossForeignCurrencyTranslationAdjustmentNetOfTax',
       'AccumulatedOtherComprehensiveIncomeLossNetOfTax',
       'AdditionalPaidInCapital',
       'AdjustmentsRelatedToTaxWithholdingForShareBasedCompensation',
       ...
       'UnrecognizedTaxBenefitsDecreasesResultingFromSettlementsWithTaxingAuthorities',
       'UnrecognizedTaxBenefitsIncomeTaxPenaltiesAndInterestAccrued',
       'UnrecognizedTaxBenefitsIncreasesResultingFromCurrentPeriodTaxPositions',
       'UnrecognizedTaxBenefitsIncreasesResultingFromPriorPeriodTaxPositions',
       'UnrecognizedTaxBenefitsInterestOnIncomeTaxesAccrued',
       'UnrecognizedTaxBenefitsReductionsResultingFromLapseOfApplicableStatuteOfLimitations',
       'UnrecognizedTaxBenefitsThatWouldImpactEff

In [7]:
sorted_transposed_df = processed_df.sort_index(ascending=False).T
sorted_transposed_df

Year,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008
fact,,,,,,,,,,,,,,,
Period End,2023-01-29 00:00:00,2022-01-30 00:00:00,2021-01-31 00:00:00,2020-02-02 00:00:00,2019-02-03 00:00:00,2018-01-28 00:00:00,2017-01-29 00:00:00,2016-01-29 00:00:00,2015-02-01 00:00:00,2014-02-02 00:00:00,2013-02-03 00:00:00,2012-01-29 00:00:00,2011-01-30 00:00:00,2010-01-31 00:00:00,2009-02-01 00:00:00
Filing Date,2023-03-24 00:00:00,2022-03-28 00:00:00,2021-03-30 00:00:00,2020-03-27 00:00:00,2019-04-04 00:00:00,2018-03-29 00:00:00,2017-03-30 00:00:00,2016-03-31 00:00:00,2015-04-02 00:00:00,2014-04-03 00:00:00,2013-04-04 00:00:00,2012-03-29 00:00:00,2011-03-31 00:00:00,2010-04-01 00:00:00,2009-04-01 00:00:00
AccountsPayableCurrent,"508,321,000","612,512,000","542,992,000","521,235,000","526,702,000","457,144,000","453,710,000","447,412,000","397,037,000","404,791,000","259,162,000","218,329,000","227,963,000","188,241,000",NaN
AccruedIncomeTaxesCurrent,"61,204,000","79,554,000","69,476,000","22,501,000","21,461,000","56,783,000","23,245,000","67,052,000","32,488,000","49,365,000","41,849,000","22,435,000","41,997,000","48,260,000",NaN
AccruedLiabilitiesCurrent,"247,594,000","319,924,000","267,592,000","175,003,000","163,559,000","134,207,000","130,187,000","127,122,000","136,012,000","138,181,000","120,632,000","111,774,000","122,440,000","107,710,000",NaN
AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment,"2,154,009,000","2,062,335,000","1,953,975,000","1,883,350,000","1,812,767,000","1,686,829,000","1,586,039,000","1,454,307,000","1,386,465,000","1,382,475,000","1,282,905,000","1,199,805,000","1,127,493,000",NaN,NaN
AccumulatedOtherComprehensiveIncomeLossForeignCurrencyTranslationAdjustmentNetOfTax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"-2,548,000","6,524,000","13,633,000","12,590,000","12,990,000","10,387,000",NaN
AccumulatedOtherComprehensiveIncomeLossNetOfTax,"-13,809,000","-10,828,000","-7,117,000","-14,587,000","-11,073,000","-6,782,000","-9,903,000","-10,616,000","-2,548,000",NaN,NaN,NaN,NaN,NaN,NaN
AdditionalPaidInCapital,"573,117,000","600,942,000","638,375,000","605,822,000","581,900,000","562,814,000","556,928,000","541,307,000","527,261,000","522,595,000","503,616,000","478,720,000","466,885,000","448,848,000",NaN


In [8]:
def generate_reverse_mapping_and_rename(df, mapping_dict):
    reverse_mapping = {}
    for new_col, old_cols in mapping_dict.items():
        for old_col in old_cols:
            suffix = old_col.split("_")[-1].lower()
            reverse_mapping[suffix] = new_col

    # Rename the columns in a case-insensitive manner
    new_columns = []
    for col in df.columns:
        col_lower = col.lower()
        if col_lower in reverse_mapping:
            new_columns.append((reverse_mapping[col_lower], col))
        else:
            new_columns.append((col, ""))

    renamed_df = df.copy()
    renamed_df.columns = pd.MultiIndex.from_tuples(new_columns)

    return renamed_df


def get_changed_columns(statement_mapping_dict, aggregated_df):
    renamed_df = generate_reverse_mapping_and_rename(
        aggregated_df, statement_mapping_dict
    )
    original_columns = set(aggregated_df.columns)
    new_columns = set([col[0] for col in renamed_df.columns])
    changed_columns = new_columns - original_columns
    changed_df = renamed_df.loc[
        :, [col for col in renamed_df.columns if col[0] in changed_columns]
    ]
    changed_df.index.name = None
    changed_df.columns.names = [None, "Gaap Name"]

    return changed_df

In [12]:
income_df = get_changed_columns(income_mapping, processed_df).sort_index(
    ascending=False
)

In [10]:
balance_df = get_changed_columns(balance_mapping, processed_df).sort_index(
    ascending=False
)
balance_df.T.sort_index(ascending=True)

,,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008
,Gaap Name,,,,,,,,,,,,,,,
Accounts Payable,AccountsPayableCurrent,"508,321,000","612,512,000","542,992,000","521,235,000","526,702,000","457,144,000","453,710,000","447,412,000","397,037,000","404,791,000","259,162,000","218,329,000","227,963,000","188,241,000",NaN
Accounts Receivable,ReceivablesNetCurrent,"115,685,000","131,683,000","143,728,000","111,737,000","107,102,000","90,119,000","88,803,000","79,304,000","67,465,000","60,330,000","62,985,000","45,961,000","41,565,000","44,187,000",NaN
Accrued Expenses,AccruedLiabilitiesCurrent,"247,594,000","319,924,000","267,592,000","175,003,000","163,559,000","134,207,000","130,187,000","127,122,000","136,012,000","138,181,000","120,632,000","111,774,000","122,440,000","107,710,000",NaN
Accumulated Depreciation PPE,AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment,"2,154,009,000","2,062,335,000","1,953,975,000","1,883,350,000","1,812,767,000","1,686,829,000","1,586,039,000","1,454,307,000","1,386,465,000","1,382,475,000","1,282,905,000","1,199,805,000","1,127,493,000",NaN,NaN
Accumulated Other Comprehensive Income(Loss),AccumulatedOtherComprehensiveIncomeLossNetOfTax,"-13,809,000","-10,828,000","-7,117,000","-14,587,000","-11,073,000","-6,782,000","-9,903,000","-10,616,000","-2,548,000",NaN,NaN,NaN,NaN,NaN,NaN
Additional Paid in Capital,AdditionalPaidInCapital,"573,117,000","600,942,000","638,375,000","605,822,000","581,900,000","562,814,000","556,928,000","541,307,000","527,261,000","522,595,000","503,616,000","478,720,000","466,885,000","448,848,000",NaN
Cash,CashAndCashEquivalentsAtCarryingValue,"367,344,000","850,338,000","1,200,337,000","432,162,000",NaN,"390,136,000",NaN,"193,647,000","222,927,000","330,121,000","424,555,000","502,757,000","628,403,000","513,943,000","148,822,000"
Cash and Restricted Cash,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents,"367,344,000","850,338,000","1,200,337,000","432,162,000","338,954,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Common Shares Authorized,CommonStockSharesAuthorized,"253,125,000","253,125,000","253,125,000","253,125,000","253,125","253,125,000","253,125,000","253,125,000","253,125,000","253,125,000","253,125,000","253,125,000","253,125,000","253,125,000",NaN


In [11]:
cash_df = get_changed_columns(cash_mapping, processed_df).sort_index(ascending=False)
cash_df.T.sort_index(ascending=True)

2022  \
                                                   Gaap Name                                                           
Amoritization of Deferred Lease Incentive          AmortizationOfLeaseIncentives                                 NaN   
Assets Acquired Under Operating Lease              RightOfUseAssetObtainedInExchangeForOperatingLe...    389,494,000   
Borrowings Under Revolving Credit Facility         ProceedsFromLinesOfCredit                                       0   
Cash Paid for Income Tax Net                       IncomeTaxesPaidNet                                    400,776,000   
Cash Paid for Interest                             InterestPaid                                                  NaN   
                                                   InterestPaidNet                                           788,000   
Cash and Cash Equivalents at Begining of Year      CashAndCashEquivalentsAtCarryingValue                 367,344,000   
                                                   CashCashEquivalentsRestrictedCashAndRestrictedC...    367,344,000   
Changes in Accounts Payable                        IncreaseDecreaseInAccountsPayable                    -113,521,000   
Changes in Accounts Receivable and Other Reciva... IncreaseDecreaseInAccountsAndOtherReceivables         -15,687,000   
Changes in Accrued Expenses                        IncreaseDecreaseInAccruedLiabilities                          NaN   
                                                   IncreaseDecreaseInAccruedLiabilitiesAndOtherOpe...    -61,995,000   
Changes in Deferred Revenue                        IncreaseDecreaseInDeferredRevenue                      31,839,000   
Changes in Income Taxes Payable                    IncreaseDecreaseInAccruedIncomeTaxesPayable           -18,231,000   
Changes in Inventory                               IncreaseDecreaseInInventories                         208,908,000   
Changes in Operating Lease Liability               IncreaseDecreaseInOperatingLeaseLiability            -242,855,000   
                                                   OperatingLeasePayments                                  2,300,000   
Changes in Prepaid Expenses and other assets       IncreaseDecreaseInOtherOperatingAssets                        NaN   
                                                   IncreaseDecreaseInPrepaidDeferredExpenseAndOthe...     11,823,000   
Changes in Restricted Cash                         IncreaseDecreaseInRestrictedCash                              NaN   
Charge for Future tax items                        TaxCutsAndJobsActOf2017IncomeTaxExpenseBenefit                NaN   
Debt Issuance Costs                                PaymentsOfDebtIssuanceCosts                                     0   
Deferred Income Taxes                              DeferredIncomeTaxExpenseBenefit                       -23,823,000   
Depreciation and Amortization                      DepreciationDepletionAndAmortization                  214,153,000   
Effect of Exchange Rates on Cash and Cash Equiv... EffectOfExchangeRateOnCashAndCashEquivalents                  NaN   
                                                   EffectOfExchangeRateOnCashCashEquivalentsRestri...     -3,188,000   
Income loss from equity method investments         IncomeLossFromEquityMethodInvestments                         NaN   
Loss on Disposal/Impairment                        AssetImpairmentCharges                                        NaN   
                                                   GainLossOnSaleOfPropertyPlantEquipment                        NaN   
                                                   GainLossOnSalesOfAssetsAndAssetImpairmentCharges      -25,116,000   
Loss related to Litigation settlement              GainLossRelatedToLitigationSettlement                         NaN   
Net Cash Provided by Operating Activities          NetCashProvidedByUsedInOperatingActivities          1,052,822,000   
                                